# 1. Read Cleaned Dataset of OpenWeb Text and ChatGPT3.5 Text

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ChatGPT = []
# Read all the jsonl files in the path
for i in range(8):
    a = pd.read_json(f"dataset/opengpttext-clean/chatgpt/urlsf_subset0{i}.jsonl", lines=True)
    ChatGPT.append(a)
ChatGPT[0].head()

,uid,text
0,[urlsf_subset00]-[83],The National Weather Service's Mike McFarland ...
1,[urlsf_subset00]-[89],The President of the United States was seen on...
2,[urlsf_subset00]-[390],Enner Valencia scored two goals in Ecuador's 2...
3,[urlsf_subset00]-[457],"Beginning with the introduction, the author sh..."
4,[urlsf_subset00]-[458],Mexico has implemented its newest data retenti...


# 2. The first method we proposed is to leverage the pretrained RoBERTa model [10] to extract relevant features from the input text

In [2]:
import torch
from transformers import RobertaTokenizer, RobertaModel

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Extract features from the text
SampleTextDF = ChatGPT[0].sample(100)
def extract_features(text):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)
    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    features = last_hidden_states[0][0].detach().numpy()
    return features

SampleTextDF['features'] = SampleTextDF['text'].apply(extract_features)
SampleTextDF.head()

In [11]:
# Extract features from the text
SampleTextDF = ChatGPT[0].sample(100)
def extract_features(text):
    tokenized_text = tokenizer.encode(text, truncation=True, max_length=512)

    # Padding if necessary
    if len(tokenized_text) < 512:
        padding_length = 512 - len(tokenized_text)
        tokenized_text.extend([0] * padding_length)

    input_ids = torch.tensor(tokenized_text).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)
    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    print(f"Shape of last_hidden_states: {last_hidden_states.shape}")  # Print shape for debugging
    TClassification = last_hidden_states[:, 0, :]  # Take the first token's representation
    features = last_hidden_states[0][0].detach().numpy()
    return features, TClassification

SampleTextDF['features'], SampleTextDF['T[CLS]'] = SampleTextDF['text'].apply(extract_features)
SampleTextDF.head()

Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of last_hidden_states: torch.Size([1, 512, 768])
Shape of l

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 12582912 bytes.

In [8]:
SampleTextDF['features'].values[2].shape

(768,)